# Magentic Workflow with Human Plan Review

## Overview

This notebook demonstrates **interactive plan review** in Magentic workflows - enabling human oversight and modification of the orchestrator's execution plan before proceeding. This pattern is essential for:

1. **Quality Assurance**: Verify AI-generated plans before expensive execution
2. **Compliance**: Ensure workflows follow organizational policies
3. **Risk Management**: Human approval for high-stakes operations
4. **Iterative Refinement**: Collaborate with AI to optimize plans

### Key Concepts:

1. **MagenticPlanReviewRequest**: Orchestrator requests human plan approval
2. **RequestInfoEvent**: Checkpoint-based human interaction mechanism
3. **MagenticPlanReviewReply**: Human response with approval/modifications
4. **Plan Modification Loop**: Iterative plan refinement cycle
5. **Streaming Integration**: Real-time plan visibility

### Plan Review Lifecycle:

```
User Task
    ↓
Orchestrator Generates Plan
    ↓
Request Plan Review (Checkpoint)
    ↓
┌─────────────────────────┐
│  Human Reviews Plan     │
│  ├── Approve            │
│  ├── Reject & Modify    │
│  └── Request Revisions  │
└─────────────────────────┘
    ↓
Resume with Feedback
    ↓
Execute (if approved) OR Regenerate (if rejected)
    ↓
Final Result
```

### Review Response Options:

| Response Type | Action | Use Case |
|--------------|--------|----------|
| **Approve** | `approved=True` | Plan is acceptable, proceed |
| **Reject** | `approved=False` | Regenerate plan with feedback |
| **Modify** | `approved=False, message=...` | Provide specific changes |

## Prerequisites

- OpenAI API key configured: `OPENAI_API_KEY` environment variable
- Agent Framework installed: `pip install agent-framework`
- Special models for agents:
  - ResearcherAgent: `gpt-4o-search-preview`
  - CoderAgent: OpenAI Assistants with code interpreter

## Setup and Imports

In [ ]:
import asyncio
import logging

import os
from dotenv import load_dotenv
from agent_framework import (
    ChatAgent,
    HostedCodeInterpreterTool,
    MagenticAgentDeltaEvent,
    MagenticAgentMessageEvent,
    MagenticBuilder,
    MagenticCallbackEvent,
    MagenticCallbackMode,
    MagenticFinalResultEvent,
    MagenticOrchestratorMessageEvent,
    MagenticPlanReviewReply,
    MagenticPlanReviewRequest,
    MagenticWorkflow,
    RequestInfoEvent,
    WorkflowOutputEvent,
)
from agent_framework.openai import OpenAIChatClient, OpenAIResponsesClient

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Load environment variables from .env file
load_dotenv('../../.env')


## Create Specialized Agents

Same multi-agent setup:
- **ResearcherAgent**: Web search and information gathering
- **CoderAgent**: Code execution and quantitative analysis

In [ ]:
async def run_plan_review_workflow() -> None:
    researcher_agent = ChatAgent(
        name="ResearcherAgent",
        description="Specialist in research and information gathering",
        instructions=(
            "You are a Researcher. You find information without additional computation or quantitative analysis."
        ),
        chat_client=OpenAIChatClient(model_id="gpt-4o-search-preview"),
    )

    coder_agent = ChatAgent(
        name="CoderAgent",
        description="A helpful assistant that writes and executes code to process and analyze data.",
        instructions="You solve questions using code. Please provide detailed analysis and computation process.",
        chat_client=OpenAIResponsesClient(),
        tools=HostedCodeInterpreterTool(),
    )

## Define Enhanced Event Callback

The callback:
1. Displays all workflow events
2. **Detects plan review requests** (RequestInfoEvent with MagenticPlanReviewRequest)
3. Signals when human input is needed
4. Captures plan details for review

In [ ]:
    last_stream_agent_id: str | None = None
    stream_line_open: bool = False

    async def on_event(event: MagenticCallbackEvent) -> None:
        """
        Process workflow events including plan review requests.
        """
        nonlocal last_stream_agent_id, stream_line_open

        if isinstance(event, MagenticOrchestratorMessageEvent):
            print(f"\n[ORCH:{event.kind}]\n\n{getattr(event.message, 'text', '')}\n{'-' * 26}")

        elif isinstance(event, MagenticAgentDeltaEvent):
            if last_stream_agent_id != event.agent_id or not stream_line_open:
                if stream_line_open:
                    print()
                print(f"\n[STREAM:{event.agent_id}]: ", end="", flush=True)
                last_stream_agent_id = event.agent_id
                stream_line_open = True
            print(event.text, end="", flush=True)

        elif isinstance(event, MagenticAgentMessageEvent):
            if stream_line_open:
                print(" (final)")
                stream_line_open = False
                print()
            msg = event.message
            if msg is not None:
                response_text = (msg.text or "").replace("\n", " ")
                print(f"\n[AGENT:{event.agent_id}] {msg.role.value}\n\n{response_text}\n{'-' * 26}")

        elif isinstance(event, MagenticFinalResultEvent):
            print("\n" + "=" * 50)
            print("FINAL RESULT:")
            print("=" * 50)
            if event.message is not None:
                print(event.message.text)
            print("=" * 50)

        # Plan review detection
        elif isinstance(event, RequestInfoEvent):
            if isinstance(event.request, MagenticPlanReviewRequest):
                print("\n" + "!" * 50)
                print("PLAN REVIEW REQUESTED")
                print("!" * 50)
                print(f"Request ID: {event.request.request_id}")
                print(f"Checkpoint ID: {event.checkpoint_id}")
                print("\nGenerated Plan:")
                print("-" * 50)
                # Plan details would be in the request object
                print("(Plan content available in request object)")
                print("-" * 50)
                print("!" * 50)

## Build Workflow with Plan Review

Standard Magentic workflow - plan review is triggered by orchestrator's planning phase.

In [ ]:
    print("\nBuilding Magentic Workflow with Plan Review...")

    workflow: MagenticWorkflow = (
        MagenticBuilder()
        .participants(researcher=researcher_agent, coder=coder_agent)
        .on_event(on_event, mode=MagenticCallbackMode.STREAMING)
        .with_standard_manager(
            chat_client=OpenAIChatClient(),
            max_round_count=10,
            max_stall_count=3,
            max_reset_count=2,
        )
        .build()
    )

## Define Complex Task for Review

In [ ]:
    task = (
        "I am preparing a report on the energy efficiency of different machine learning model architectures. "
        "Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 "
        "on standard datasets. Then, estimate the CO2 emissions associated with each, assuming training "
        "on an Azure Standard_NC6s_v3 VM for 24 hours. Provide tables for clarity, and recommend the most "
        "energy-efficient model per task type."
    )

    print(f"\nTask: {task}")
    print("\nStarting workflow execution...")

## Execute Until Plan Review Request

Run workflow until it requests plan approval.

In [ ]:
    try:
        checkpoint_id: str | None = None
        plan_request: MagenticPlanReviewRequest | None = None

        print("\n[PHASE 1: Execute until plan review]\n")

        async for event in workflow.run_stream(task):
            print(event)

            # Capture plan review request
            if isinstance(event, RequestInfoEvent):
                if isinstance(event.request, MagenticPlanReviewRequest):
                    plan_request = event.request
                    checkpoint_id = event.checkpoint_id
                    print(f"\n✓ Plan review checkpoint created: {checkpoint_id}")
                    break  # Pause for human review

        if checkpoint_id is None or plan_request is None:
            print("\n⚠ No plan review was requested - workflow may not be configured for plan review.")
            return

## Simulate Human Plan Review

### Review Process:

1. **Examine generated plan**: Review orchestrator's proposed approach
2. **Make decision**: Approve, reject, or request modifications
3. **Provide feedback**: Specific guidance for plan adjustments

### Example Scenarios:

#### Scenario 1: Approval
```python
response = MagenticPlanReviewReply(
    approved=True,
    message="Plan looks good. Please proceed."
)
```

#### Scenario 2: Modification Request
```python
response = MagenticPlanReviewReply(
    approved=False,
    message="Please add comparison with Vision Transformers and include inference latency analysis."
)
```

#### Scenario 3: Rejection
```python
response = MagenticPlanReviewReply(
    approved=False,
    message="This approach won't work. Focus only on inference energy, not training."
)
```

In [ ]:
        print("\n" + "=" * 50)
        print("HUMAN PLAN REVIEW")
        print("=" * 50)
        print("\nReviewing orchestrator's plan...")
        print("\nDecision: APPROVE with minor suggestions")
        print("=" * 50 + "\n")

        # Human provides approval with suggestions
        human_response = MagenticPlanReviewReply(
            approved=True,
            message=(
                "Plan approved. Please ensure the comparison includes both training and inference metrics, "
                "and provide recommendations with justifications based on the energy efficiency data."
            ),
        )

        print(f"Human feedback: {human_response.message}")
        print(f"Approval status: {'APPROVED' if human_response.approved else 'REJECTED'}\n")

## Resume Workflow with Human Feedback

### Resume Behavior:

- **If approved**: Orchestrator proceeds with plan execution
- **If rejected**: Orchestrator regenerates plan considering feedback
- **Feedback integration**: Human message guides next steps

The orchestrator incorporates human feedback into its decision-making.

In [ ]:
        print("\n[PHASE 2: Resume with human feedback]\n")

        output: str | None = None
        async for event in workflow.run_stream_from_checkpoint(
            checkpoint_id=checkpoint_id,
            responses={plan_request.request_id: human_response},
        ):
            print(event)
            if isinstance(event, WorkflowOutputEvent):
                output = str(event.data)

        if output is not None:
            print(f"\nWorkflow completed with result:\n\n{output}")
        else:
            print("\nWorkflow completed without output.")

    except Exception as e:
        print(f"Workflow execution failed: {e}")
        import traceback
        traceback.print_exc()

## Run the Complete Plan Review Workflow

In [ ]:
await run_plan_review_workflow()

## Expected Output Pattern

```
Building Magentic Workflow with Plan Review...

Task: I am preparing a report on the energy efficiency...

[PHASE 1: Execute until plan review]

[ORCH:planning]
Generating execution plan...
--------------------------

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
PLAN REVIEW REQUESTED
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Request ID: req_xyz789
Checkpoint ID: checkpoint_abc123

Generated Plan:
--------------------------------------------------
(Plan content available in request object)
--------------------------------------------------
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

✓ Plan review checkpoint created: checkpoint_abc123

==================================================
HUMAN PLAN REVIEW
==================================================

Reviewing orchestrator's plan...

Decision: APPROVE with minor suggestions
==================================================

Human feedback: Plan approved. Please ensure...
Approval status: APPROVED

[PHASE 2: Resume with human feedback]

[ORCH:executing]
Proceeding with approved plan (incorporating feedback)...
--------------------------

[STREAM:ResearcherAgent]: Researching energy consumption data...

[AGENT:ResearcherAgent] assistant
Found training and inference metrics for all three models...
--------------------------

[STREAM:CoderAgent]: Calculating emissions and creating tables...

==================================================
FINAL RESULT:
==================================================
Energy Efficiency Report with justified recommendations...
==================================================
```

## Key Takeaways

### 1. Plan Review Configuration

No special configuration needed - standard Magentic orchestrator supports plan review:

```python
workflow = (
    MagenticBuilder()
    .participants(...)
    .on_event(callback, mode=MagenticCallbackMode.STREAMING)
    .with_standard_manager(...)  # Plan review is built-in
    .build()
)
```

### 2. Detecting Plan Review Requests

```python
async for event in workflow.run_stream(task):
    if isinstance(event, RequestInfoEvent):
        if isinstance(event.request, MagenticPlanReviewRequest):
            # Plan review requested
            plan_request = event.request
            checkpoint_id = event.checkpoint_id
            break  # Pause for human input
```

### 3. Constructing Review Responses

#### Approval
```python
from agent_framework import MagenticPlanReviewReply

response = MagenticPlanReviewReply(
    approved=True,
    message="Plan looks good. Proceed with execution."
)
```

#### Rejection with Feedback
```python
response = MagenticPlanReviewReply(
    approved=False,
    message="Please modify the plan to include X, Y, and Z."
)
```

### 4. Resuming with Response

```python
async for event in workflow.run_stream_from_checkpoint(
    checkpoint_id=checkpoint_id,
    responses={plan_request.request_id: human_response}
):
    # Workflow continues based on approval status
    if isinstance(event, WorkflowOutputEvent):
        result = event.data
```

### 5. Iterative Plan Refinement

If plan is rejected, orchestrator may regenerate and request review again:

```python
review_iterations = 0
max_iterations = 3

while review_iterations < max_iterations:
    async for event in workflow.run_stream(...):
        if isinstance(event, RequestInfoEvent):
            if isinstance(event.request, MagenticPlanReviewRequest):
                # Another review iteration
                review_iterations += 1
                human_response = get_human_review()  # Get new feedback
                # Resume with feedback
                break
```

### 6. Plan Review vs. Other Human-in-Loop Patterns

| Pattern | Timing | Purpose | Frequency |
|---------|--------|---------|----------|
| **Plan Review** | Before execution | Validate approach | Once per workflow |
| **Data Approval** | Mid-execution | Verify intermediate results | Multiple times |
| **Tool Approval** | Before tool use | Security/compliance | Per tool call |
| **Final Review** | After execution | Quality check | Once at end |

### 7. Production Implementation

#### Web Interface
```python
from fastapi import FastAPI, BackgroundTasks

app = FastAPI()

@app.post("/workflow/start")
async def start_workflow(task: str):
    # Start workflow, pause at plan review
    checkpoint_id = await execute_until_plan_review(task)
    return {"checkpoint_id": checkpoint_id, "status": "awaiting_review"}

@app.post("/workflow/review")
async def submit_review(checkpoint_id: str, approved: bool, message: str):
    response = MagenticPlanReviewReply(approved=approved, message=message)
    result = await resume_workflow(checkpoint_id, response)
    return {"result": result}
```

#### Notification System
```python
async def notify_reviewers(plan_request: MagenticPlanReviewRequest):
    # Send email/Slack notification
    await send_notification(
        to="reviewers@company.com",
        subject="Plan Review Required",
        body=f"Review plan for request {plan_request.request_id}"
    )
```

### 8. Security and Compliance

#### Audit Logging
```python
def log_plan_review(
    plan_request: MagenticPlanReviewRequest,
    reviewer: str,
    decision: MagenticPlanReviewReply
) -> None:
    audit_log.info({
        "event": "plan_review",
        "request_id": plan_request.request_id,
        "reviewer": reviewer,
        "approved": decision.approved,
        "feedback": decision.message,
        "timestamp": datetime.now().isoformat()
    })
```

#### Role-Based Access
```python
def verify_reviewer_permission(user_id: str, plan_request: MagenticPlanReviewRequest) -> bool:
    # Check if user has permission to review this plan
    return user_id in authorized_reviewers_for_task(plan_request.task_type)
```

### 9. Use Cases

#### High-Stakes Operations
- Financial transactions
- Data deletion or modification
- External API calls
- Model deployments

#### Compliance Requirements
- HIPAA-compliant healthcare workflows
- Financial regulatory compliance
- Legal document processing
- Security-sensitive operations

#### Quality Assurance
- Research workflows requiring peer review
- Content generation with editorial oversight
- Data analysis with validation gates
- Report generation with approval

#### Cost Optimization
- Verify expensive LLM-heavy plans
- Approve large-scale data processing
- Review resource-intensive operations

### 10. Best Practices

#### Clear Plan Presentation
- Extract and format plan details clearly
- Show estimated costs/resources
- Display agent assignments
- Highlight risks or assumptions

#### Structured Feedback
- Provide specific, actionable feedback
- Reference plan steps by number
- Suggest concrete modifications
- Explain rationale for rejection

#### Timeout Handling
```python
import asyncio

async def get_human_review_with_timeout(
    plan_request: MagenticPlanReviewRequest,
    timeout_seconds: int = 3600
) -> MagenticPlanReviewReply:
    try:
        return await asyncio.wait_for(
            get_human_review(plan_request),
            timeout=timeout_seconds
        )
    except asyncio.TimeoutError:
        # Default action on timeout
        return MagenticPlanReviewReply(
            approved=False,
            message="Review timeout - automatically rejected for safety"
        )
```

### 11. Comparison with Checkpointing

| Feature | Plan Review | Generic Checkpointing |
|---------|------------|----------------------|
| **Purpose** | Validate plan before execution | Resume from any point |
| **Timing** | Pre-execution | Anytime during execution |
| **Frequency** | Typically once | Multiple times |
| **Decision** | Approve/reject plan | Provide any input |
| **Impact** | Changes execution approach | Continues execution |

### 12. Advanced Patterns

#### Multi-Level Approval
```python
# Manager reviews plan
manager_response = MagenticPlanReviewReply(approved=True, message="Approved")

# Continue execution
async for event in workflow.run_stream_from_checkpoint(...):
    # Later: Senior manager reviews intermediate results
    if isinstance(event, RequestInfoEvent):
        senior_response = get_senior_manager_approval()
        # Resume with additional approval
```

#### Conditional Review
```python
def should_require_review(plan: dict) -> bool:
    # Review only high-cost or high-risk plans
    estimated_cost = calculate_plan_cost(plan)
    return estimated_cost > COST_THRESHOLD
```

#### Collaborative Review
```python
async def get_consensus_review(
    plan_request: MagenticPlanReviewRequest,
    reviewers: list[str]
) -> MagenticPlanReviewReply:
    # Collect reviews from multiple people
    reviews = await asyncio.gather(*[
        get_individual_review(plan_request, reviewer)
        for reviewer in reviewers
    ])
    
    # Require unanimous approval
    all_approved = all(r.approved for r in reviews)
    combined_feedback = "\n".join(r.message for r in reviews)
    
    return MagenticPlanReviewReply(
        approved=all_approved,
        message=combined_feedback
    )
```